# SEIR Generalize model with rho as an step function
#Author: Imelda Trejo 
#November: 25,2021

In [7]:
# Import relevant modules
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from scipy.stats import lognorm
from scipy.stats import gamma
from scipy.stats import weibull_min
from scipy.stats import nbinom
from scipy.interpolate import BSpline
from matplotlib import cm
import datetime

In [8]:
plt.rcParams.update({'font.size':16})
colors = cm.plasma(np.linspace(0,1,12))
dateIndex=np.load("dateIndex.npy",allow_pickle=True).ravel() #Incidence data
#print(dateIndex)
n=np.size(dateIndex)
print(n)
print(dateIndex[-4:])

Yaux=np.load("Hidalgo.npy")  #Incidence data
print(len(Yaux))

1021
['Dec-09-22' 'Dec-10-22' 'Dec-11-22' 'Dec-12-22']
1021


#Functions that does not depend on data

In [9]:
#---------Polynomial spline
def Pspline(t,vk,c,k,q0):
    n = len(vk) - k
    assert (n >= k+1) and (len(c) >= n)
    return sum(c[i] * BaseSpline(t, k, i, vk,q0) for i in range(n))   
#---------mixing-monotonic-and-standar-Base
def BaseSpline(t,k,i,T,q0): #k0,,t0=6--->90
    n = len(t)
    b=np.zeros(n) ##base function of degree k-1
    if i<=q0:
        aux=BSpline.basis_element(T[i+1:i+k+1],extrapolate=False) #degree k-2
        a=aux.integrate(T[i+1],T[i+k+1])
        for j in range(0,tp):                                 #M-spline degree k-1
            b[j]=1-aux.integrate(T[1],t[j])/a
    else:
        b= BSpline.basis_element(vk[i:i+k+1],extrapolate=False)(t) #degree k-1
    return np.nan_to_num(b)

#Reproductive Number, trapezoidal rule approx, constant projection
def RepNumber(Q,psi):
    tp=len(psi)
    A,Rt=np.zeros((tp-1,tp-1)),np.zeros(tp)
    for k in range(0,tp-1): #drop the last element 
        A[k,0:tp-k-1]=Q[k+1:tp]   
        A[k,tp-k-1:]=Q[tp-1]*np.ones(k)    
    Rt[0:tp-1]=h*(2*np.dot(A,psi[1:tp])+psi[0]*Q[0])/2
    Rt[tp-1]=Rt[tp-2] 
    for k in range(0,tp-1):
        if Q[k]==0:
            #print(Rt[k])
            Rt[k]=0
    return Rt

In [13]:
key=0

state=["HIDALGO",'PUEBLA','MICHOACAN','MEXICO','FL','MI','Lin','LinPsi1','LinPsi2','LinPsi3'][key]

prefix=state

parLog = np.genfromtxt(prefix+'-parLog.txt')
Yaux=np.load("%s.npy"%state)  #Incidence data
n=len(Yaux)

In [14]:


#-----------------------Define the partition for the epidecmis time observed [0, tn]  
h,m=0.1,10
tp=int(n/h)+1            #total points
t=np.zeros(tp)
for i in range(0,tp):
    t[i]=h*i  
#------------------------Base-spline interpolation----with partition (0,T)={0,xhi_1,...,xhi_q,T}
ls=21                    #length between the k-nots 
q=int(n/ls)+1         
vk=np.zeros(q)
for i in range(1,q-1):
    vk[i]=ls*i
vk[q-1]=n
#-------------------------------boundary-knots
vk=np.append([-3*ls,-2*ls,-ls],vk)
vk=np.append(vk,[n+ls,n+2*ls,n+3*ls])
#--------------Wigths
q=q+2  #base elements
print('Dimention of the Base-spline',q)    
#-------------------Bsplines functions: derivatives and gram matrix
q0=8                  #number of monotonic base functions
#print(vk)

#--------------------Infectious period functions------#1-----------
##laten distribution
Faux=np.load("F.npy")
Gaux=np.load("Gaux.npy") #1-G(t)
Psi=np.load("Psi.npy")

F=Faux[0:tp]   
G=Gaux[0:tp]      
psi=Psi[0:tp] 


Dimention of the Base-spline 51


In [19]:
#--------------------------Data set extraction

#Par=np.load('NMPar.npy') 

Par = np.genfromtxt('HIDALGO-parLog.txt')

offset = len(Par)

#sampled-size=sz
sz=100
newparLog=Par[0:-sz,]

rho,Rt=np.zeros((sz,tp)),np.zeros((sz,tp))    

for k in range(0,sz):
    rho[k]=np.exp(Pspline(t,vk,newparLog[k,0:q],4,q0)) #rho
    Rt[k]=RepNumber(rho[k],psi)

R=Par[:,q]
rNM=np.around(np.exp(np.median(R)),decimals=3)
rhoL,rhoU,rhoM=np.zeros(tp),np.zeros(tp),np.zeros(tp)   #uper lower bounds, median
RtNML,RtNMU,RtNMM=np.zeros(tp),np.zeros(tp),np.zeros(tp)   #uper lower bounds, median

for i in range(0,tp):
    rhoL[i]=np.around(np.quantile(rho[:,i],0.025),decimals=5)
    rhoU[i]=np.around(np.quantile(rho[:,i],.975),decimals=5)    
    rhoM[i]=np.around(np.median(rho[:,i]),decimals=5)        
    RtNML[i]=np.around(np.quantile(Rt[:,i],0.025),decimals=5)
    RtNMU[i]=np.around(np.quantile(Rt[:,i],.975),decimals=5)    
    RtNMM[i]=np.around(np.median(Rt[:,i]),decimals=5)    
    



Par = np.genfromtxt('PUEBLA-parLog.txt')
newparLog=Par[0:-sz,]

rho,Rt=np.zeros((sz,tp)),np.zeros((sz,tp))    
for k in range(0,sz):
    rho[k]=np.exp(Pspline(t,vk,newparLog[k,0:q],4,q0)) #rho
    Rt[k]=RepNumber(rho[k],psi)
R=Par[:,q]
rTX=np.around(np.exp(np.median(R)),decimals=3)
rhoL,rhoU,rhoM=np.zeros(tp),np.zeros(tp),np.zeros(tp)   #uper lower bounds, median
RtTXL,RtTXU,RtTXM=np.zeros(tp),np.zeros(tp),np.zeros(tp)   #uper lower bounds, median

for i in range(0,tp):
    rhoL[i]=np.around(np.quantile(rho[:,i],0.025),decimals=5)
    rhoU[i]=np.around(np.quantile(rho[:,i],.975),decimals=5)    
    rhoM[i]=np.around(np.median(rho[:,i]),decimals=5)        
    RtTXL[i]=np.around(np.quantile(Rt[:,i],0.025),decimals=5)
    RtTXU[i]=np.around(np.quantile(Rt[:,i],.975),decimals=5)    
    RtTXM[i]=np.around(np.median(Rt[:,i]),decimals=5)    
    



In [ ]:
#Read Raw Data
#title='Texas'  #'New York'  #'Texas'  'Michigan'
titleCA,tinitCA,tfinalCA='California','Jan-25-20','2022-16-19' #CA 
titleTX,tinitTX,tfinalTX='Texas','Feb-12-20','2022-16-19' #TX
titleNY,tinitNY,tfinalNY='New York','Mar-01-20','2022-16-19' #NY
titleFL,tinitFL,tfinalFL='Florida','Mar-01-20','2022-16-30' #FL
titleMI,tinitMI,tfinalMI='Michigan','Mar-10-20','2022-16-19' #MI
titleNM,tinitNM,tfinalNM='New Mexico','Mar-11-20','2022-16-19' #NM

In [ ]:
#t1s,t2s='Apr-11-20', 'Jun-10-20'
t1s,t2s='Apr-05-20', 'Jun-14-20'


t3s,t4s='Jun-04-21', 'Aug-23-21'

t5s,t6s='Nov-11-21', 'Jan-15-22'

In [ ]:
##Delta
t1=10*(np.where(dateIndex==t1s)[0][0])
t2=10*(np.where(dateIndex==t2s)[0][0])
t3=10*(np.where(dateIndex==t3s)[0][0])
t4=10*(np.where(dateIndex==t4s)[0][0])
##Omicron
t5=10*(np.where(dateIndex==t5s)[0][0])
t6=10*(np.where(dateIndex==t6s)[0][0])

#print(dateIndex,t1)
#print(init)


ID=['New Mexico','Texas','New York','California','Florida','Michigan','Lin','Lin','Lin','Lin']

In [ ]:
#Rt=1
R1=np.ones(tp)
axisyU=5 #upper bound for axis-y
colorFix2=0
colorFix=4
colorFix3=9
alphaFix=0.1
alphaFix2=0.3
fig, ax = plt.subplots(6,1,figsize=(17,20), gridspec_kw={'height_ratios': [1,1,1,1,1,1]})

#
ax[0].plot(t,R1,'--',lw=3,color = 'k')
ax[0].fill_between(t,RtLCA,RtRCA,color ='b',alpha=0.2)

ax[0].fill_between(t[t1:t2],0*R1[t1:t2], axisyU*R1[t1:t2],color =colors[colorFix2],alpha=alphaFix)
ax[0].fill_between(t[t1:t2],0*R1[t1:t2], axisyU*R1[t1:t2],color ='skyblue',alpha=alphaFix2)

ax[0].fill_between(t[t3:t4],0*R1[t3:t4], axisyU*R1[t3:t4],color =colors[colorFix],alpha=alphaFix)
ax[0].fill_between(t[t5:t6],0*R1[t5:t6], axisyU*R1[t5:t6],color =colors[colorFix3],alpha=alphaFix)



ax[0].plot(t,RtCA,color ='b',lw=3, label='$R_t$ posterior median',alpha=1)
init=np.where(dateIndex==tinitCA)[0][0]
#ax[0].axvline(x=init,color=colors[2],linestyle=':',linewidth=4,label='first reported cases')
ax[0].legend(loc=2, frameon=False)
#ax[0].set_ylabel('$R_t$',fontsize=18)
#ax[1].legend(loc='lower left') 
ax[0].legend(loc='upper center',frameon=False,fontsize=18) 
ax[0].set_title('%s' %ID[3],fontsize=20)
ax[0].set_ylim(0,axisyU)
ax[0].grid(axis='x')
ax[0].grid(axis='y')
ax[0].set_xticks(range(0,n,20))
ax[0].set_xticklabels(dateIndex[::20], rotation=90)




ax[1].fill_between(t[t1:t2],0*R1[t1:t2], axisyU*R1[t1:t2],color =colors[colorFix2],alpha=alphaFix)
ax[1].fill_between(t[t1:t2],0*R1[t1:t2], axisyU*R1[t1:t2],color ='skyblue',alpha=alphaFix2)

ax[1].fill_between(t[t3:t4],0*R1[t3:t4], axisyU*R1[t3:t4],color =colors[colorFix],alpha=alphaFix)
ax[1].fill_between(t[t5:t6],0*R1[t5:t6], axisyU*R1[t5:t6],color =colors[colorFix3],alpha=alphaFix)



ax[1].plot(t,R1,'--',lw=3,color = 'k')
ax[1].fill_between(t,RtLTX,RtRTX,color ='b',alpha=0.2)
ax[1].plot(t,RtTX,color ='b',lw=3, label='$R_t$ posterior median',alpha=1)
init=np.where(dateIndex==tinitTX)[0][0]
#ax[1].axvline(x=init,color=colors[2],linestyle=':',linewidth=4,label='first reported cases')
ax[1].legend(loc=2, frameon=False)
#ax[1].set_ylabel('$R_t$',fontsize=18)
ax[1].legend(loc='best') 
#ax[1].legend(loc='lower left') 
ax[1].legend(loc='upper center',frameon=False,fontsize=18) 
ax[1].set_title('%s' %ID[1],fontsize=20)
ax[1].set_ylim(0,axisyU)
ax[1].grid(axis='x')
ax[1].grid(axis='y')
ax[1].set_xticks(range(0,n,20))
ax[1].set_xticklabels(dateIndex[::20], rotation=90)



ax[2].fill_between(t[t1:t2],0*R1[t1:t2], axisyU*R1[t1:t2],color =colors[colorFix2],alpha=alphaFix)
ax[2].fill_between(t[t1:t2],0*R1[t1:t2], axisyU*R1[t1:t2],color ='skyblue',alpha=alphaFix2)

ax[2].fill_between(t[t3:t4],0*R1[t3:t4], axisyU*R1[t3:t4],color =colors[colorFix],alpha=alphaFix)
ax[2].fill_between(t[t5:t6],0*R1[t5:t6], axisyU*R1[t5:t6],color =colors[colorFix3],alpha=alphaFix)


ax[2].plot(t,R1,'--',lw=3,color = 'k')
ax[2].fill_between(t,RtLNY,RtRNY,color ='b',alpha=0.2)
ax[2].plot(t,RtNY,color ='b',lw=3, label='$R_t$ posterior median',alpha=1)
init=np.where(dateIndex==tinitNY)[0][0]
#ax[2].axvline(x=init,color=colors[2],linestyle=':',linewidth=4,label='first reported cases')
ax[2].legend(loc=2, frameon=False)
#ax[2].set_ylabel('$R_t$',fontsize=18)
ax[2].legend(loc='best') 
#ax[1].legend(loc='lower left') 
ax[2].legend(loc='upper center',frameon=False,fontsize=18) 
ax[2].set_title('%s' %ID[2],fontsize=20)
ax[2].set_ylim(0,axisyU)
ax[2].grid(axis='x')
ax[2].grid(axis='y')
ax[2].set_xticks(range(0,n,20))
ax[2].set_xticklabels(dateIndex[::20], rotation=90)


#fl 4

ax[3].fill_between(t[t1:t2],0*R1[t1:t2], axisyU*R1[t1:t2],color =colors[colorFix2],alpha=alphaFix)
ax[3].fill_between(t[t1:t2],0*R1[t1:t2], axisyU*R1[t1:t2],color ='skyblue',alpha=alphaFix2)

ax[3].fill_between(t[t3:t4],0*R1[t3:t4], axisyU*R1[t3:t4],color =colors[colorFix],alpha=alphaFix)
ax[3].fill_between(t[t5:t6],0*R1[t5:t6], axisyU*R1[t5:t6],color =colors[colorFix3],alpha=alphaFix)



ax[3].plot(t,R1,'--',lw=3,color = 'k')
ax[3].fill_between(t,RtLFL,RtRFL,color ='b',alpha=0.2)
ax[3].plot(t,RtFL,color ='b',lw=3, label='$R_t$ posterior median',alpha=1)
init=np.where(dateIndex==tinitFL)[0][0]
#ax[3].axvline(x=init,color=colors[2],linestyle=':',linewidth=4,label='first reported cases')
ax[3].legend(loc=2, frameon=False)
#ax[3].set_ylabel('$R_t$',fontsize=18)
#ax[1].legend(loc='lower left') 
ax[3].legend(loc='upper center',frameon=False,fontsize=18) 
ax[3].set_title('%s' %ID[4],fontsize=20)
ax[3].set_ylim(0,axisyU)
ax[3].grid(axis='x')
ax[3].grid(axis='y')
ax[3].set_xticks(range(0,n,20))
ax[3].set_xticklabels(dateIndex[::20], rotation=90)


#Mi 5

ax[4].fill_between(t[t1:t2],0*R1[t1:t2], axisyU*R1[t1:t2],color =colors[colorFix2],alpha=alphaFix)
ax[4].fill_between(t[t1:t2],0*R1[t1:t2], axisyU*R1[t1:t2],color ='skyblue',alpha=alphaFix2)

ax[4].fill_between(t[t3:t4],0*R1[t3:t4], axisyU*R1[t3:t4],color =colors[colorFix],alpha=alphaFix)
ax[4].fill_between(t[t5:t6],0*R1[t5:t6], axisyU*R1[t5:t6],color =colors[colorFix3],alpha=alphaFix)



ax[4].plot(t,R1,'--',lw=3,color = 'k')
ax[4].fill_between(t,RtLMI,RtRMI,color ='b',alpha=0.2)
ax[4].plot(t,RtMI,color ='b',lw=3, label='$R_t$ posterior median',alpha=1)
init=np.where(dateIndex==tinitMI)[0][0]
#ax[4].axvline(x=init,color=colors[2],linestyle=':',linewidth=4,label='first reported cases')
ax[4].legend(loc=2, frameon=False)
#ax[4].set_ylabel('$R_t$',fontsize=18)
ax[4].legend(loc='upper center') 
#ax[1].legend(loc='lower left') 
ax[4].legend(loc='upper center',frameon=False,fontsize=18) 
ax[4].set_title('%s' %ID[5],fontsize=20)
ax[4].set_ylim(0,axisyU)
ax[4].grid(axis='x')
ax[4].grid(axis='y')
ax[4].set_xticks(range(0,n,20))
ax[4].set_xticklabels(dateIndex[::20], rotation=90)

fig.autofmt_xdate()



ax[5].fill_between(t[t1:t2],0*R1[t1:t2], axisyU*R1[t1:t2],color =colors[colorFix2],alpha=alphaFix)
ax[5].fill_between(t[t1:t2],0*R1[t1:t2], axisyU*R1[t1:t2],color ='skyblue',alpha=alphaFix2)

ax[5].fill_between(t[t3:t4],0*R1[t3:t4], axisyU*R1[t3:t4],color =colors[colorFix],alpha=alphaFix)
ax[5].fill_between(t[t5:t6],0*R1[t5:t6], axisyU*R1[t5:t6],color =colors[colorFix3],alpha=alphaFix)



ax[5].plot(t,R1,'--',lw=3,color = 'k')
ax[5].fill_between(t,RtLNM,RtRNM,color ='b',alpha=.2)
ax[5].plot(t,RtNM,color ='b',lw=3, label='$R_t$ posterior median',alpha=1)
init=np.where(dateIndex==tinitNM)[0][0]
#ax[5].axvline(x=init,color=colors[2],linestyle=':',linewidth=4,label='first reported cases')
ax[5].legend(loc=2, frameon=False)
#ax[5].set_ylabel('$R_t$',fontsize=18)
ax[5].legend(loc='best') 
#ax[1].legend(loc='lower left') 
ax[5].legend(loc='upper center',frameon=False,fontsize=18) 
ax[5].set_title('%s' %ID[0],fontsize=20)
ax[5].set_ylim(0,axisyU)
ax[5].grid(axis='x')
ax[5].grid(axis='y')
ax[5].set_xticks(range(0,n,20))
ax[5].set_xticklabels(dateIndex[::20], rotation=70)
plt.tick_params(labelsize=15)
fig.tight_layout()

fig.savefig('RtALLnew.pdf')
